In [6]:
!pip install -U sentence-transformers
!pip install -r ./bertscore/requirements.txt

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Cloning https://github.com/huggingface/evaluate (to revision a4bdc10c48a450b978d91389a48dbb5297835c7d) to /private/var/folders/6m/zj0yj2tx1kdb2y65vxrkptt80000gn/T/pip-req-build-tgq4ua_x
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/evaluate /private/var/folders/6m/zj0yj2tx1kdb2y65vxrkptt80000gn/T/pip-req-build-tgq4ua_x
  Running command git rev-parse -q --verify 'sha^a4bdc10c48a450b978d91389a48dbb5297835c7d'
  Running command git fetch -q https://github.com/huggingface/evaluate a4bdc10c48a450b978d91389a48dbb5297835c7d
  Resolved https://github.com/huggingface/evaluate to commit a4bdc10c48a450b978d91389a48dbb5297835c7d
  Preparing metadata (setup.py) ... done


In [7]:
import pandas as pd
from evaluate import load
from sentence_transformers import SentenceTransformer
import numpy as np

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


root_path = "./data/eval-"

gemma = pd.read_csv(root_path + "gemma.csv")
Llama2 = pd.read_csv(root_path + "Llama2.csv")
llama3 = pd.read_csv(root_path + "llama3.csv")
Mistral = pd.read_csv(root_path + "Mistral.csv")

model_eval_list = [gemma, Llama2, llama3, Mistral]

gemma

,joke,explanation,zero-shot,one-shot,five-shot
0,Q: How do you make a fire with two sticks? A: ...,The joke plays on the dual meaning of the word...,The humor in this joke lies in the unexpected ...,The joke highlights the unexpected use of a ma...,The joke creates humor through wordplay. The a...
1,Q: Where do roses sleep? A: In a flower bed!,The joke plays on the double meaning of the ph...,"The joke is funny because it pun on the word ""...",The joke relies on the multiple meanings of th...,The joke plays on the multiple meanings of the...
2,Q: Why did the kids fire their squirt guns int...,The joke plays on the double meaning of the ph...,The joke is funny because it plays on the mult...,The joke's humor stems from the unexpected and...,The joke plays on the multiple meanings of the...
3,Q: Where do generals keep their armies? A: In ...,This joke plays on the double meaning of the w...,The humor in this joke lies in the unexpected ...,The joke relies on the unexpected and humorous...,The joke plays on the multiple meanings of the...
4,Q: Why are graveyards noisy? A: Because of all...,The joke plays on the dual meaning of the word...,The joke is funny because it playfully suggest...,The humor in this joke stems from the unexpect...,The joke plays on the multiple meanings of the...
5,Q: What do you call a fossil that never does a...,The joke plays on the dual meaning of 'bones.'...,The joke creates humor through unexpected word...,The joke creates humor through the unexpected ...,The joke plays on the multiple meanings of the...
6,Q: What is it called when a cat wins a dog sho...,The joke hinges on a wordplay involving the pu...,The joke is funny because it pun on the words ...,The joke plays on the play on words between 'c...,The joke plays on the humorous combination of ...
7,Q: How does an Eskimo child fix his toys? A: W...,"The joke plays on the word 'igloo,' which is a...",The joke is funny because it puns on the word ...,The joke relies on the multiple meanings of th...,The joke is funny because it plays on the mult...
8,Q: Why did the invisible man turn down a job o...,The joke plays on the double meaning of the ph...,The joke is funny because it plays on the conc...,The joke relies on the unexpected and humorous...,The joke plays on the invisibility of the prot...
9,Q: What did the ol’ pirate say on his birthday...,The joke plays on the phonetic similarity betw...,The joke is funny because it plays on the ster...,The joke plays on the stereotype of pirates an...,The joke relies on the multiple meanings of th...


In [8]:
def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1 * norm_vec2)

def metric_cal(row):
    metric_dict = dict()

    explanation = row['explanation']

    bertscore = load("bertscore")
    predict_keys = ['zero-shot', 'one-shot', 'five-shot']
    metrics = ['precision', 'recall', 'f1', 'cos-sim-1', 'cos-sim-2']

    references = [explanation]
    for predict in predict_keys:
        predict_dict = dict()
        predictions = [row[predict]]
        results = bertscore.compute(predictions=predictions, references=references, model_type="distilbert-base-uncased", nthreads=12)
        
        for metric in metrics:
            if metric in results.keys():
                predict_dict[metric] = results[metric]
            else:
                if metric == 'cos-sim-1':
                # using two sentence transformer to calculate cosine similarity
                    model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
                elif metric == 'cos-sim-2':
                    model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

                embeddings_original = model.encode(explanation)
                embeddings_predict = model.encode(predict)
                predict_dict[metric] = cosine_similarity(embeddings_original, embeddings_predict)
        metric_dict[predict] = predict_dict

    return metric_dict

In [10]:
bertscore = load("bertscore")
predicts = ["zero-shot", "one-shot", "five-shot"]
metrics = ['precision', 'recall', 'f1', 'cos-sim-1', 'cos-sim-2']
model_name_list = ['gemma', 'Llama2', 'llama3', 'Mistral']
result_dict = dict()

for i in range(len(model_name_list)):
    model = model_eval_list[i]
    print("model: ", model_name_list[i])
    # model_metric_dict = dict()
    for index, row in model.iterrows():
        # print("model: ", index)
        metric_dict = metric_cal(row)

        for predict in predicts:
            for metric in metrics:
                model.at[index, predict+"-"+metric] = metric_dict[predict][metric]
                
    for predict in predicts:
        for metric in metrics:
            print(predict, " Avg ",metric,": ",model[predict+"-"+metric].mean())

model:  gemma
zero-shot  Avg  precision :  0.8464875638484954
zero-shot  Avg  recall :  0.7982853198051453
zero-shot  Avg  f1 :  0.8215682411193848
zero-shot  Avg  cos-sim-1 :  0.09988323760218919
zero-shot  Avg  cos-sim-2 :  0.06326074155978859
one-shot  Avg  precision :  0.8488240206241607
one-shot  Avg  recall :  0.8124789834022522
one-shot  Avg  f1 :  0.8301099860668182
one-shot  Avg  cos-sim-1 :  0.1383771537989378
one-shot  Avg  cos-sim-2 :  0.07614214895293117
five-shot  Avg  precision :  0.8647567415237427
five-shot  Avg  recall :  0.8326711809635162
five-shot  Avg  f1 :  0.8482455599308014
five-shot  Avg  cos-sim-1 :  0.10869395297020673
five-shot  Avg  cos-sim-2 :  0.08161911707429681
model:  Llama2
zero-shot  Avg  precision :  0.8248942649364471
zero-shot  Avg  recall :  0.8206189382076263
zero-shot  Avg  f1 :  0.8226209950447082
zero-shot  Avg  cos-sim-1 :  0.09988323760218919
zero-shot  Avg  cos-sim-2 :  0.06326074155978859
one-shot  Avg  precision :  0.8369956958293915
on